In [89]:
# This Python 3 environment comes with many helpful analytics libraries installed 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import string
from string import digits
import matplotlib.pyplot as plt
%matplotlib inline
import re

import seaborn as sns
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.layers import Input, LSTM, Embedding, Dense
from keras.models import Model

print(os.listdir("../input"))

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', -1)

# Any results you write to the current directory are saved as output.

['Hindi_English_Truncated_Corpus.csv']


In [90]:
lines=pd.read_csv("../input/Hindi_English_Truncated_Corpus.csv",encoding='utf-8')

In [91]:
lines['source'].value_counts()

tides        50000
ted          39881
indic2012    37726
Name: source, dtype: int64

In [92]:
lines=lines[lines['source']=='ted']

In [93]:
lines.head(20)

,source,english_sentence,hindi_sentence
0,ted,politicians do not have permission to do what needs to be done.,"राजनीतिज्ञों के पास जो कार्य करना चाहिए, वह करने कि अनुमति नहीं है ."
1,ted,"I'd like to tell you about one such child,","मई आपको ऐसे ही एक बच्चे के बारे में बताना चाहूंगी,"
3,ted,what we really mean is that they're bad at not paying attention.,हम ये नहीं कहना चाहते कि वो ध्यान नहीं दे पाते
7,ted,"And who are we to say, even, that they are wrong",और हम होते कौन हैं यह कहने भी वाले कि वे गलत हैं
13,ted,So there is some sort of justice,तो वहाँ न्याय है
23,ted,This changed slowly,धीरे धीरे ये सब बदला
26,ted,were being produced.,उत्पन्न नहीं कि जाती थी.
30,ted,"And you can see, this LED is going to glow.","और जैसा आप देख रहे है, ये एल.ई.डी. जल उठेगी।"
32,ted,"to turn on the lights or to bring him a glass of water,","लाईट जलाने के लिए या उनके लिए पानी लाने के लिए,"
35,ted,Can you imagine saying that?,क्या आप ये कल्पना कर सकते है


In [94]:
pd.isnull(lines).sum()

source              0
english_sentence    0
hindi_sentence      0
dtype: int64

In [95]:
lines=lines[~pd.isnull(lines['hindi_sentence'])]

In [96]:
lines.drop_duplicates(inplace=True)

* ### Let us pick any 25000 rows from the dataset.

In [97]:
lines=lines.sample(n=25000,random_state=42)
lines.shape

(25000, 3)

In [98]:
# Lowercase all characters
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.lower())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.lower())

In [99]:
# Remove quotes
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub("'", '', x))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub("'", '', x))

In [100]:
exclude = set(string.punctuation) # Set of all special characters
# Remove all the special characters
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))
lines['english_sentence']=lines['english_sentence'].apply(lambda x: ''.join(ch for ch in x if ch not in exclude))

In [101]:
# Remove all numbers from text
remove_digits = str.maketrans('', '', digits)
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.translate(remove_digits))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.translate(remove_digits))

lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: re.sub("[२३०८१५७९४६]", "", x))

# Remove extra spaces
lines['english_sentence']=lines['english_sentence'].apply(lambda x: x.strip())
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: x.strip())
lines['english_sentence']=lines['english_sentence'].apply(lambda x: re.sub(" +", " ", x))
lines['hindi_sentence']=lines['hindi_sentence'].apply(lambda x: re.sub(" +", " ", x))


In [102]:
# Add start and end tokens to target sequences
lines['english_sentence'] = lines['english_sentence'].apply(lambda x : 'START_ '+ x + ' _END')

In [103]:
lines.head()

,source,english_sentence,hindi_sentence
82040,ted,START_ we still dont know who her parents are who she is _END,हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है
85038,ted,START_ no keyboard _END,कोई कुंजीपटल नहीं
58018,ted,START_ but as far as being a performer _END,लेकिन एक कलाकार होने के साथ
74470,ted,START_ and this particular balloon _END,और यह खास गुब्बारा
122330,ted,START_ and its not as hard as you think integrate climate solutions into all of your innovations _END,और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें


In [104]:
### Get English and Hindi Vocabulary
all_eng_words=set()
for eng in lines['english_sentence']:
    for word in eng.split():
        if word not in all_eng_words:
            all_eng_words.add(word)

all_hindi_words=set()
for hin in lines['hindi_sentence']:
    for word in hin.split():
        if word not in all_hindi_words:
            all_hindi_words.add(word)

In [105]:
len(all_eng_words)

14032

In [106]:
len(all_hindi_words)

17538

In [107]:
lines['length_eng_sentence']=lines['english_sentence'].apply(lambda x:len(x.split(" ")))
lines['length_hin_sentence']=lines['hindi_sentence'].apply(lambda x:len(x.split(" ")))

In [108]:
lines.head()

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
82040,ted,START_ we still dont know who her parents are who she is _END,हम अभी तक नहीं जानते हैं कि उसके मातापिता कौन हैं वह कौन है,13,14
85038,ted,START_ no keyboard _END,कोई कुंजीपटल नहीं,4,3
58018,ted,START_ but as far as being a performer _END,लेकिन एक कलाकार होने के साथ,9,6
74470,ted,START_ and this particular balloon _END,और यह खास गुब्बारा,6,4
122330,ted,START_ and its not as hard as you think integrate climate solutions into all of your innovations _END,और जितना आपको लगता है यह उतना कठिन नहीं हैअपने सभी नवाचारों में जलवायु समाधान को एकीकृत करें,18,18


In [109]:
lines[lines['length_hin_sentence']>30].shape

(0, 5)

In [110]:
lines=lines[lines['length_eng_sentence']<=20]
lines=lines[lines['length_hin_sentence']<=20]

In [111]:
lines.shape

(24907, 5)

In [112]:
print("maximum length of Hindi Sentence ",max(lines['length_hin_sentence']))
print("maximum length of English Sentence ",max(lines['length_eng_sentence']))

maximum length of Hindi Sentence  20
maximum length of English Sentence  20


In [113]:
max_length_src=max(lines['length_hin_sentence'])
max_length_tar=max(lines['length_eng_sentence'])

In [114]:
input_words = sorted(list(all_hindi_words))
target_words = sorted(list(all_eng_words))
num_encoder_tokens = len(all_hindi_words)
num_decoder_tokens = len(all_eng_words)
num_encoder_tokens, num_decoder_tokens

(17538, 14032)

In [115]:
num_decoder_tokens += 1 #for zero padding


In [116]:
input_token_index = dict([(word, i+1) for i, word in enumerate(input_words)])
target_token_index = dict([(word, i+1) for i, word in enumerate(target_words)])

In [117]:
reverse_input_char_index = dict((i, word) for word, i in input_token_index.items())
reverse_target_char_index = dict((i, word) for word, i in target_token_index.items())

In [118]:
lines = shuffle(lines)
lines.head(10)

,source,english_sentence,hindi_sentence,length_eng_sentence,length_hin_sentence
9626,ted,START_ recovering old poems _END,याद आ रही है एक पुरानी कविता,5,7
65547,ted,START_ so we worked with a company _END,तो हमने एक कंपनी के साथ काम किया,8,8
28999,ted,START_ you see three different structures being built by these robots _END,इन रोबोट को तीन अलग संरचनाओं को बनाते हुए,12,9
7668,ted,START_ whether its a bailout an oil spill _END,चाहे वो कंपनी को उबारना हो तेल का रिसाव हो,9,10
87013,ted,START_ its not just throwing them out at random _END,आखिर इन्हें बिना किसी उद्द्देश्य के यूँ ही तो नहीं फैला दिया गया होगा,10,14
40327,ted,START_ to continue putting polluting facilities in my neighborhood _END,जो कि मेरे आसपास प्रदूषण उत्पन्न कर रहे हैं।,10,9
86643,ted,START_ if i let her go by herself and we get in the car _END,अगर मैंने माँ को अकेले जाने दिया और हम गाडी में बैठ गए,15,13
109348,ted,START_ its the friends the allies _END,दोस्त और सहयोगी,7,3
53783,ted,START_ and then takes a picture of whatever is in front of you _END,और यह आपके सामने जो भी है उसकी फ़ोटो खींच लेती है,14,12
20327,ted,START_ to export goods dutyfree to the european union market _END,बिना कर के माल निर्यात का मौका दिया है,11,9


### Split the data into train and test

In [119]:
X, y = lines['hindi_sentence'], lines['english_sentence']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2,random_state=42)
X_train.shape, X_test.shape

((19925,), (4982,))

### Let us save this data

In [120]:
X_train.to_pickle('X_train.pkl')
X_test.to_pickle('X_test.pkl')


In [121]:
def generate_batch(X = X_train, y = y_train, batch_size = 128):
    ''' Generate a batch of data '''
    while True:
        for j in range(0, len(X), batch_size):
            encoder_input_data = np.zeros((batch_size, max_length_src),dtype='float32')
            decoder_input_data = np.zeros((batch_size, max_length_tar),dtype='float32')
            decoder_target_data = np.zeros((batch_size, max_length_tar, num_decoder_tokens),dtype='float32')
            for i, (input_text, target_text) in enumerate(zip(X[j:j+batch_size], y[j:j+batch_size])):
                for t, word in enumerate(input_text.split()):
                    encoder_input_data[i, t] = input_token_index[word] # encoder input seq
                for t, word in enumerate(target_text.split()):
                    if t<len(target_text.split())-1:
                        decoder_input_data[i, t] = target_token_index[word] # decoder input seq
                    if t>0:
                        # decoder target sequence (one hot encoded)
                        # does not include the START_ token
                        # Offset by one timestep
                        decoder_target_data[i, t - 1, target_token_index[word]] = 1.
            yield([encoder_input_data, decoder_input_data], decoder_target_data)

### Encoder-Decoder Architecture

In [122]:
latent_dim=300

In [123]:
# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb =  Embedding(num_encoder_tokens, latent_dim, mask_zero = True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

In [124]:
# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(num_decoder_tokens, latent_dim, mask_zero = True)
dec_emb = dec_emb_layer(decoder_inputs)
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

In [125]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

In [126]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
input_6 (InputLayer)            (None, None)         0                                            
__________________________________________________________________________________________________
embedding_5 (Embedding)         (None, None, 300)    5261400     input_5[0][0]                    
__________________________________________________________________________________________________
embedding_6 (Embedding)         (None, None, 300)    4209900     input_6[0][0]                    
__________________________________________________________________________________________________
lstm_5 (LS

In [127]:
train_samples = len(X_train)
val_samples = len(X_test)
batch_size = 128
epochs = 100

In [128]:
model.fit_generator(generator = generate_batch(X_train, y_train, batch_size = batch_size),
                    steps_per_epoch = train_samples//batch_size,
                    epochs=epochs,
                    validation_data = generate_batch(X_test, y_test, batch_size = batch_size),
                    validation_steps = val_samples//batch_size)



Epoch 1/100
155/155 [==============================] - 52s 336ms/step - loss: 6.1812 - val_loss: 5.8792
Epoch 2/100
155/155 [==============================] - 49s 317ms/step - loss: 5.6327 - val_loss: 5.5977
Epoch 3/100
155/155 [==============================] - 50s 323ms/step - loss: 5.3024 - val_loss: 5.3851
Epoch 4/100
155/155 [==============================] - 50s 323ms/step - loss: 5.0518 - val_loss: 5.2718
Epoch 5/100
155/155 [==============================] - 50s 323ms/step - loss: 4.8413 - val_loss: 5.1762
Epoch 6/100
155/155 [==============================] - 50s 322ms/step - loss: 4.6478 - val_loss: 5.1155
Epoch 7/100
155/155 [==============================] - 50s 321ms/step - loss: 4.4712 - val_loss: 5.0641
Epoch 8/100
155/155 [==============================] - 49s 318ms/step - loss: 4.3066 - val_loss: 5.0195
Epoch 9/100
155/155 [==============================] - 50s 322ms/step - loss: 4.1497 - val_loss: 5.0097
Epoch 10/100
155/155 [==============================] - 50s 320m

155/155 [==============================] - 50s 323ms/step - loss: 0.1408 - val_loss: 7.3467
Epoch 80/100
155/155 [==============================] - 50s 322ms/step - loss: 0.1349 - val_loss: 7.3507
Epoch 81/100
155/155 [==============================] - 50s 321ms/step - loss: 0.1300 - val_loss: 7.3705
Epoch 82/100
155/155 [==============================] - 50s 322ms/step - loss: 0.1236 - val_loss: 7.3695
Epoch 83/100
155/155 [==============================] - 50s 321ms/step - loss: 0.1193 - val_loss: 7.3840
Epoch 84/100
155/155 [==============================] - 50s 321ms/step - loss: 0.1169 - val_loss: 7.3940
Epoch 85/100
155/155 [==============================] - 50s 326ms/step - loss: 0.1137 - val_loss: 7.4148
Epoch 86/100
155/155 [==============================] - 50s 325ms/step - loss: 0.1079 - val_loss: 7.4233
Epoch 87/100
155/155 [==============================] - 51s 328ms/step - loss: 0.1042 - val_loss: 7.4498
Epoch 88/100
155/155 [==============================] - 51s 332ms/st

In [129]:
model.save_weights('nmt_weights.h5')

In [130]:
# Encode the input sequence to get the "thought vectors"
encoder_model = Model(encoder_inputs, encoder_states)

# Decoder setup
# Below tensors will hold the states of the previous time step
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

dec_emb2= dec_emb_layer(decoder_inputs) # Get the embeddings of the decoder sequence

# To predict the next word in the sequence, set the initial states to the states from the previous time step
decoder_outputs2, state_h2, state_c2 = decoder_lstm(dec_emb2, initial_state=decoder_states_inputs)
decoder_states2 = [state_h2, state_c2]
decoder_outputs2 = decoder_dense(decoder_outputs2) # A dense softmax layer to generate prob dist. over the target vocabulary

# Final decoder model
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs2] + decoder_states2)


In [131]:
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)
    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1,1))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0] = target_token_index['START_']

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += ' '+sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '_END' or
           len(decoded_sentence) > 50):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [132]:
train_gen = generate_batch(X_train, y_train, batch_size = 1)
k=-1


In [133]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: आपने कई बार ये सुना होगा
Actual English Translation:  youve heard that saying 
Predicted English Translation:  youve heard that saying 


In [134]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: मेसेज भेजते समय
Actual English Translation:  while textmessaging 
Predicted English Translation:  while textmessaging 


In [135]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: और यह सब क्या मानव स्वभाव के बारे में हमें क्या बताता है
Actual English Translation:  and what all of this can tell us about human nature 
Predicted English Translation:  and what all of this can tell us about human le


In [136]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: चूहा एक पूरा जीव है आखिर में
Actual English Translation:  the rat is an entire organism after all 
Predicted English Translation:  the rat is an entire organism after all 


In [137]:
k+=1
(input_seq, actual_output), _ = next(train_gen)
decoded_sentence = decode_sequence(input_seq)
print('Input Hindi sentence:', X_train[k:k+1].values[0])
print('Actual English Translation:', y_train[k:k+1].values[0][6:-4])
print('Predicted English Translation:', decoded_sentence[:-4])

Input Hindi sentence: केवल बडे खेतों में इस्तेमाल के लिये ही बनायी गयी थी।
Actual English Translation:  and they were constructed for fields that were too large 
Predicted English Translation:  and they were constructed for fields that were
